In [1]:
############################################################################
# Copyright (c) 2022, NVIDIA CORPORATION.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
############################################################################

### Note: once you have downloaded the dataset, put it in the `data` folder

In [3]:
data_path = "./data"
input_data_path = data_path+"/card_transaction.v1.csv"

In [1]:
# untar the downloaded dataset
!tar -xzf data/transactions.tgz; mv *.csv data

tar (child): data/transactions.tgz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now
mv: cannot stat '*.csv': No such file or directory


In [1]:
# iso18245 is a library that helps us to clean the merchant category code (MCC) column in the dataset
%pip install iso18245

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import numpy as np
import pandas as pd
import iso18245

In [4]:
# Read Tabformer dataset
# Dataset can be downloaded from: https://ibm.ent.box.com/v/tabformer-data

data = pd.read_csv(input_data_path)
data.head()

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
0,0,0,2002,9,1,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,NaN,No
1,0,0,2002,9,1,06:42,$38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
2,0,0,2002,9,2,06:22,$120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
3,0,0,2002,9,2,17:45,$128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754.0,5651,NaN,No
4,0,0,2002,9,3,06:23,$104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750.0,5912,NaN,No


In [5]:
data.shape

(24386900, 15)

# Clean the dataset

In [6]:
# column names to lowercase, remove punctuation
data.columns = data.columns.str.lower()
data.columns = data.columns.str.replace(' ','_')
data.columns = data.columns.str.replace('?', '')

/tmp/ipykernel_1134/1044835818.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data.columns = data.columns.str.replace('?', '')


In [7]:
data.head()

,user,card,year,month,day,time,amount,use_chip,merchant_name,merchant_city,merchant_state,zip,mcc,errors,is_fraud
0,0,0,2002,9,1,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,NaN,No
1,0,0,2002,9,1,06:42,$38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
2,0,0,2002,9,2,06:22,$120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
3,0,0,2002,9,2,17:45,$128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754.0,5651,NaN,No
4,0,0,2002,9,3,06:23,$104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750.0,5912,NaN,No


In [8]:
def _prepare_data(df):
    
    # split time and save as two new cols: hour and minute
    df[['hour', 'minute']] = df.time.str.split(':', expand=True)

    df.hour = df.hour.astype(int)
    df.minute = df.minute.astype(int)

    # add date col
    df['date'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute']])

    # sort rows by date and re-order the cols
    df = df.sort_values('date')\
          [['user', 'card', 'date', 'year', 'month', 'day', 'time', 'hour', 'minute', 'amount',
            'use_chip', 'merchant_name', 'merchant_city', 'merchant_state', 'zip',
            'mcc', 'errors', 'is_fraud']]\
         .reset_index(drop=True)

    # factorize is_fraud col into 1 and 0.
    df['is_fraud'], fraud_key = df['is_fraud'].factorize()

    # remove out the $ sign in the amount column and convert to float.
    df['amount'] = df.amount.str.strip('$').astype(float)

    df['use_chip'] = df['use_chip'].str.strip()
    df['merchant_city'] = df['merchant_city'].str.strip()

    # Remove nulls - Zip code is null when international transactions
    df['zip'] = df['zip'].fillna(-1)
    df['zip'] = df['zip'].astype(int)
    df['zip'] = df['zip'].astype(str)
    df['zip'] = df['zip'].replace('-1', '')
    df['merchant_state'] = df['merchant_state'].fillna('')

    # Factorize use_chip categorical feature
    df['use_chip'], unq_use_chip = df['use_chip'].factorize()

    # Remove nans in strings
    df['zip'] = df.zip.astype(str)
    # fill nans with empty string...
    df['zip'] = df['zip'].fillna('')
    df['merchant_state'] = df['merchant_state'].fillna('')

    # Errors column
    unq_errors = ['Technical Glitch', 'Bad Zipcode', 'Bad CVV', 'Bad PIN', 'Bad Card Number', 
                  'Bad Expiration', 'Insufficient Balance']
    exploded = df['errors'].str.strip(',').str.split(',').explode()
    raw_one_hot = pd.get_dummies(exploded.astype(pd.CategoricalDtype(categories=unq_errors)), columns=["errors"])
    errs = raw_one_hot.groupby(raw_one_hot.index).sum()
    df = pd.concat([df, errs], axis=1)
    df = df.rename(columns={col:f'errors_{col}' for col in unq_errors})

    # Add num_cards_per_user feature
    num_cards_per_user = df[['user', 'card']].groupby('user').card.nunique()
    num_cards_per_user = num_cards_per_user.rename('num_cards_per_user')
    df = pd.merge(df, num_cards_per_user, on='user', how='left')

    # Add day of the week feature
    df['dayofweek'] = pd.to_datetime(df.date).dt.dayofweek

    # Factorize merchant names since min/max of their names (which is an int) is at the bounds of 64 bit int
    df['merchant_name'], merch_name_uniq = df['merchant_name'].factorize()
    df['merchant_city'], merc_city_uniq = df['merchant_city'].factorize()
    df['merchant_state'], merc_state_uniq = df['merchant_state'].factorize()
    df['zip'], zip_uniq = df['zip'].factorize()

    # range from 0 to 29 inclusive
    # for inference, will need to add back 1991 years.
    min_year = df.year.min()
    df['year'] = df.year-df.year.min()

    # encode all real mccs
    all_mccs_info = iso18245.get_all_mccs()
    all_mccs = sorted([int(mcc.mcc) for mcc in all_mccs_info])
    df['mcc'] = df['mcc'].map({all_mccs[i]: i for i in range(len(all_mccs))})

    mappings = {'unq_use_chip': unq_use_chip,
            'merch_name_uniq': merch_name_uniq,
            'merc_city_uniq': merc_city_uniq,
            'merc_state_uniq': merc_state_uniq,
            'zip_uniq': zip_uniq,
            'all_mccs': all_mccs,
           }
    
    
    num_cols = sorted(['amount'])
    cat_cols = sorted(['card', 'num_cards_per_user', 'year', 'month', 'day', 'hour', 'minute', 'use_chip', 
                   'merchant_name', 'merchant_city', 'merchant_state', 'zip', 'mcc', 
                   'errors_Bad CVV', 'errors_Bad Card Number', 'errors_Bad Expiration', 'errors_Bad PIN',
                   'errors_Bad Zipcode', 'errors_Insufficient Balance', 'errors_Technical Glitch'
                  ])
    target_col = ['is_fraud']
    
    cards = [int(max(df[col].nunique(), df[col].max() + 1)) for col in cat_cols]
    
    meta_data = {'mappings':mappings, 'num_cols':num_cols, 'cat_cols':cat_cols, 'target_col':target_col, 'cards':cards}
    
    return df, meta_data


In [9]:
# Takes a few min to run
data, meta_data = _prepare_data(data)

In [10]:
data.head()

,user,card,date,year,month,day,time,hour,minute,amount,...,is_fraud,errors_Technical Glitch,errors_Bad Zipcode,errors_Bad CVV,errors_Bad PIN,errors_Bad Card Number,errors_Bad Expiration,errors_Insufficient Balance,num_cards_per_user,dayofweek
0,791,1,1991-01-02 07:10:00,0,1,2,07:10,7,10,68.00,...,0,0,0,0,0,0,0,0,4,2
1,791,1,1991-01-02 07:17:00,0,1,2,07:17,7,17,-68.00,...,0,0,0,0,0,0,0,0,4,2
2,791,1,1991-01-02 07:21:00,0,1,2,07:21,7,21,113.62,...,0,0,0,0,0,0,0,0,4,2
3,791,1,1991-01-02 17:30:00,0,1,2,17:30,17,30,114.73,...,0,0,0,0,0,0,0,0,4,2
4,791,1,1991-01-03 09:03:00,0,1,3,09:03,9,3,251.71,...,0,0,0,0,0,0,0,0,4,3


In [11]:
for cat, card in zip(meta_data['cat_cols'], meta_data['cards']):
    print(f'{cat}: {card}')

card: 9
day: 32
errors_Bad CVV: 2
errors_Bad Card Number: 2
errors_Bad Expiration: 2
errors_Bad PIN: 2
errors_Bad Zipcode: 2
errors_Insufficient Balance: 2
errors_Technical Glitch: 2
hour: 24
mcc: 892
merchant_city: 13429
merchant_name: 100343
merchant_state: 224
minute: 60
month: 13
num_cards_per_user: 10
use_chip: 3
year: 30
zip: 27322


In [12]:
# save meta data

import pickle 
with open(data_path+'/meta_data.pickle', 'wb') as handle:
    pickle.dump(meta_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Split dataset into 2 regions

In [16]:
# save list of states and shuffle in random order
states = data['merchant_state'].unique()

np.random.seed(100)
np.random.shuffle(states)

# split list of states into 2 groups
states_group1, states_group2 = np.array_split(states, 2)

In [17]:
# Group 1
data_group1 = data[(data['merchant_state'].isin(states_group1))]

print("Shape of group 1 data: ", data_group1.shape)
print("Unique merchants in group 1: ", data_group1['merchant_state'].nunique())

Shape of group 1 data:  (10110286, 27)
Unique merchants in group 1:  112


In [18]:
# Group 2
data_group2 = data[(data['merchant_state'].isin(states_group2))]

print("Shape of group 2 data: ", data_group2.shape)
print("Unique merchants in group 2: ", data_group2['merchant_state'].nunique())

Shape of group 2 data:  (14276614, 27)
Unique merchants in group 2:  112


# Train / test splits

In [20]:
# Group 1
train_group1 = data_group1.sample(frac = 0.8, random_state=1)
test_group1 = data_group1.drop(train_group1.index)

In [21]:
# Group 2
train_group2 = data_group2.sample(frac = 0.8, random_state=2)
test_group2 = data_group2.drop(train_group2.index)

# Save data splits

In [22]:
# Save group 1
train_group1.to_csv(data_path+"/site-1.csv", index=False)
test_group1.to_csv(data_path+"/val-site-1.csv", index=False)

In [23]:
# Save group 2
train_group2.to_csv(data_path+"/site-2.csv", index=False)
test_group2.to_csv(data_path+"/val-site-2.csv", index=False)